In [17]:
import numpy as np
import cv2
import torch
import random
import glob
import os
from rich.progress import track

%matplotlib inline

## Poisson

In [18]:
def generate_poisson_noise(img, scale=1.0, gray_noise=False):
    """Generate poisson noise.
    Ref: https://github.com/scikit-image/scikit-image/blob/main/skimage/util/noise.py#L37-L219
    Args:
        img (Numpy array): Input image, shape (h, w, c), range [0, 1], float32.
        scale (float): Noise scale. Default: 1.0.
        gray_noise (bool): Whether generate gray noise. Default: False.
    Returns:
        (Numpy array): Returned noisy image, shape (h, w, c), range[0, 1],
            float32.
    """
    if gray_noise:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # round and clip image for counting vals correctly
    img = np.clip((img * 255.0).round(), 0, 255) / 255.
    vals = len(np.unique(img))
    vals = 2**np.ceil(np.log2(vals))
    out = np.float32(np.random.poisson(img * vals) / float(vals))
    noise = out - img
    if gray_noise:
        noise = np.repeat(noise[:, :, np.newaxis], 3, axis=2)
    return noise * scale


def add_poisson_noise(img, scale=1.0, clip=True, rounds=False, gray_noise=False):
    """Add poisson noise.
    Args:
        img (Numpy array): Input image, shape (h, w, c), range [0, 1], float32.
        scale (float): Noise scale. Default: 1.0.
        gray_noise (bool): Whether generate gray noise. Default: False.
    Returns:
        (Numpy array): Returned noisy image, shape (h, w, c), range[0, 1],
            float32.
    """
    noise = generate_poisson_noise(img, scale, gray_noise)
    out = img + noise
    if clip and rounds:
        out = np.clip((out * 255.0).round(), 0, 255) / 255.
    elif clip:
        out = np.clip(out, 0, 1)
    elif rounds:
        out = (out * 255.0).round() / 255.
    return out

In [19]:
HR_path = './testset/McM/HR/'
LR_path = './testset/LR/'
id_img_paths = glob.glob(os.path.join(HR_path, '*.jpg')) + \
               glob.glob(os.path.join(HR_path, '*.png'))

scale = 2
for i in track(id_img_paths):
    out_path = LR_path + i.split('/')[-1].split('.')[0] + '.png'
    img = cv2.imread(i)    
    out = add_poisson_noise(img/255., scale=scale)*255.
    cv2.imwrite(out_path, out)

Output()

## Gaussian

In [15]:
def add_Gaussian_noise_color(img, noise_level=15):
    img += np.random.normal(0, noise_level/255.0, img.shape).astype(np.float32)
    img = np.clip(img, 0.0, 1.0)
    return img



In [16]:
HR_path = './PAtestset/HR/'
LR_path = './PAtestset/LR/'
id_img_paths = glob.glob(os.path.join(HR_path, '*.jpeg')) + \
               glob.glob(os.path.join(HR_path, '*.png'))

noise_level = 15
for i in track(id_img_paths):
    out_path = LR_path + i.split('/')[-1].split('.')[0] + '.png'
    img = cv2.imread(i)
    out = add_Gaussian_noise_color(img/255., noise_level=noise_level)*255.
    cv2.imwrite(out_path, out)

Output()